# 程式部屬

In [ ]:
group_name = "getNewsData.js"  # 自動生成程式群組
db_path = r'.\ford-news-source.db'  # 資料庫位置
project_path = r'.\Playwright' # 專案位置
file_path = r'.\Playwright\getNewsData.js' # 程式預定生成位置
output_file_rename = "getAllNewsData.js" #程式生成
output_folder = r'.\Playwright\output_autobuild' # 程式自動重建後生成位置

In [ ]:
import os
import sqlite3
import codecs
import jsbeautifier
import shutil
import codecs
from datetime import datetime, timezone


## 生成通用程式
函數生成時會組合相對應通用部分

In [ ]:

# 國際時間
def get_utc_time():
    utc_time = datetime.now(timezone.utc)
    return utc_time.strftime("%Y%m%d%H%M%S")

# 客製化String
def trim_string(string):
    return string.strip()

# 特殊join
def join_with_braces(array, type=''):
    if type:
        joined_string = ', '.join(['{' + item + '}' for item in array])
    else:
        joined_string = ', '.join(array)

    return joined_string

# 組合區段資料處理
def accumulate_data(data):
    print('accumulate_data')

    accumulated_data = {}
    prev_code_snippet = None

    for item in data:
        CODE_SNIPPET = item['CODE_SNIPPET']
        CODE = item['CODE']
        CODE_DYNAMIC = item['CODE_DYNAMIC']

        # 基於每一行的特徵額外處理
        if CODE_SNIPPET in accumulated_data:
            # 已存在則增加資料
            accumulated_data[CODE_SNIPPET].append(CODE)
        else:
            accumulated_data[CODE_SNIPPET] = [CODE]

        if CODE_DYNAMIC and prev_code_snippet != CODE_SNIPPET:
            dynamic_tag = "{CODE_DYNAMIC}"
            accumulated_data[prev_code_snippet].append(dynamic_tag)
            print(f'{CODE_SNIPPET}有動態snipple，以增加內容{dynamic_tag}至{prev_code_snippet}')

        prev_code_snippet = CODE_SNIPPET

    # 移除作為動態模組的 {CODE_DYNAMIC} 
    for CODE_SNIPPET in accumulated_data:
        code_lines = accumulated_data[CODE_SNIPPET]
        last_line = code_lines[-1]
        if last_line.strip() == dynamic_tag:
            accumulated_data[CODE_SNIPPET] = code_lines[:-1]

    return accumulated_data

# 儲存資料
def save_data_to_database(group_name, data, db_path):
    print('save_data_to_database')

    # 如果沒資料則離開
    if not data:
        return

    current_time = get_utc_time()

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    try:
        # 清理舊資料
        # cursor.execute(
        #    "DELETE FROM CODE_SNIPPET WHERE GROUP_NAME = ?", (group_name,))

        for index, (CODE_SNIPPET, CODE) in enumerate(data.items()):
            row_data = '\n'.join(CODE).strip()
            cursor.execute("INSERT INTO CODE_SNIPPET (GROUP_NAME,SNIPPET_NAME,SNIPPET_CODE,SNIPPET_DYNAMIC,SNIPPET_ORDER,DATE) VALUES (?, ?, ?, ?, ?, ?)",
                           (group_name, CODE_SNIPPET, row_data, trim_string(CODE_SNIPPET.split(',')[-1]), index, current_time))

        conn.commit()
    except Exception as e:
        print(f"Error：{e}")
    finally:
        conn.close()

# 從 CODE_STORE 取得程式碼資料
def get_data_from_database(group_name, db_path):
    print('get_data_from_database')

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    try:
        cursor.execute(
            "SELECT ID, GROUP_NAME, CODE_ROW ,CODE_SNIPPET, CODE, CODE_DYNAMIC FROM CODE_STORE WHERE GROUP_NAME = ? ORDER BY CODE_ROW ASC", (group_name,))
        rows = cursor.fetchall()

        data = []
        for row in rows:
            item = {
                'ID': row[0],
                'GROUP_NAME': row[1],
                'CODE_ROW': row[2],
                'CODE_SNIPPET': row[3],
                'CODE': row[4],
                'CODE_DYNAMIC': row[5]
            }
            data.append(item)

        conn.commit()
    except Exception as e:
        print(f"Error：{e}")
    finally:
        conn.close()

    return data


# 共用之連接資料庫方法
def connect_database(db_path):
    conn = sqlite3.connect(db_path)
    return conn

# 共用之關閉資料庫方法
def close_database(conn):
    conn.close()

# 資料寫入CODE_FUNCTION
def write_to_code_function(conn, group_name, function_name, function_version, function_code):
    cursor = conn.cursor()
    cursor.execute("INSERT INTO CODE_FUNCTION (GROUP_NAME, FUNCTION_NAME, FUNCTION_VERSION, CODE, DATE) VALUES (?, ?, ?, ?, ?)",
                   (group_name, function_name, function_version, function_code, get_utc_time()))
    conn.commit()

# 處理Snippet相關函數
def get_code_snippet_from_database(conn, group_name):
    cursor = conn.cursor()
    cursor.execute("SELECT ID, SNIPPET_DYNAMIC, COALESCE(SNIPPET_TRANSFORM, '') AS SNIPPET_TRANSFORM, SNIPPET_CODE, SNIPPET_COMMENT FROM CODE_SNIPPET WHERE GROUP_NAME = ?", (group_name,))
    rows = cursor.fetchall()
    snippet_data = []
    for row in rows:
        snippet_data.append({
            'ID': row[0],
            'SNIPPET_DYNAMIC': row[1],
            'SNIPPET_TRANSFORM': row[2],
            'SNIPPET_CODE': row[3],
            'SNIPPET_COMMENT': row[4]
        })
    return snippet_data

# 根據snippet_dynamic和snippet_transform從snippet_data取得合適snippet_code
def get_code_snippet_from_snippet_data(snippet_dynamic, snippet_transform, snippet_data):
    for snippet in snippet_data:
        if snippet['SNIPPET_DYNAMIC'] == snippet_dynamic and snippet['SNIPPET_TRANSFORM'] == snippet_transform:
            return snippet['SNIPPET_CODE'] 


## 生成客製程式
針對抓回資料差異性進行客製化生成


In [ ]:

def process_snippets1(group_name, db_path):

    code_snippet_deep = []  # 儲存層級函數

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute("SELECT ID, GROUP_NAME, CODE_ROW, CODE FROM CODE_STORE WHERE GROUP_NAME = ? ORDER BY CODE_ROW ASC", (group_name,))
    rows = cursor.fetchall()

    try:
        for row in rows:
            code = row[3]
            dynamic_snippet ='' 
            
            # 狀況有兩種，一種是Snippet標籤，一種是沒有
            if "// Snippet" in code:
                print(f'now in :{code}') 

                # 層級資料
                snippet_name = code.split(":")[1].strip() 
                
                if snippet_name.startswith(":"):
                    snippet_name = snippet_name[1:]

                # 檢查Snippet組合行
                if "Start" in code:
                    code_snippet_deep.append(snippet_name)
                    snippet_name2db = join_with_braces(code_snippet_deep)

                    print(f'code_snippet_deep:{code_snippet_deep}')
                    
                elif "End" in code and snippet_name == code_snippet_deep[-1]:
                    # 移除前先保留應塞入的snippet層級
                    snippet_name2db = join_with_braces(code_snippet_deep)
                    
                    # 若為多層程式，則應標記最底層可以作為動態處理的項目    
                    if len(code_snippet_deep) > 1 :
                        dynamic_snippet =  code_snippet_deep[-1]
                    
                    code_snippet_deep.pop() 

                    print(f'code_snippet_deep:{code_snippet_deep}')

            elif code_snippet_deep:
                # 如果有值，則取得目前層級塞入
                snippet_name2db = join_with_braces(code_snippet_deep)            

            # 若為多層程式，則應標記最底層可以作為動態處理的項目    
            if len(code_snippet_deep) > 1 :
                dynamic_snippet =  code_snippet_deep[-1]

            # 更新CODE_STORE表中对应行的CODE_SNippet字段
            cursor.execute("UPDATE CODE_STORE SET CODE_SNIPPET = ? , CODE_DYNAMIC = ? WHERE ID = ?", (snippet_name2db, dynamic_snippet, row[0]))
        
        conn.commit()

    except Exception as e:
        print(f"Error：{e}")

    finally:
        # 关闭数据库连接
        conn.close()


# JS整理後寫入資料庫 CODE_STORE
def write_code_to_database(file_path, db_path):
    try:
        # 讀取生成的JS
        with codecs.open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
            js_code = file.read()

        # 自動格式化JS
        formatted_code = jsbeautifier.beautify(js_code)

        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        file_name = os.path.basename(file_path)
        group_name = file_name

        cursor.execute("DELETE FROM CODE_STORE WHERE GROUP_NAME = ?", (group_name,))

        for line_number, line in enumerate(formatted_code.splitlines(), start=1):
            trimmed_line = trim_string(line)

            if not trimmed_line:
                continue

            current_time = get_utc_time()
            cursor.execute("INSERT OR REPLACE INTO CODE_STORE (GROUP_NAME, CODE_ROW, CODE, DATE) VALUES (?, ?, ?, ?)",
                           (group_name, line_number, trimmed_line, current_time))

        conn.commit()
        conn.close()

        print("Save")

    except Exception as e:
        print(f"Error：{e}")


# 定義增加文件到數據庫的函數
def add_group_project(project_path, file_path, db_path):
    print("進入系統")

    # 確保指定文件完整路徑下的JS文件存在，並獲取文件時間
    if os.path.exists(file_path) and file_path.endswith(".js"):
        print("開始處理資料")
        try:
            # 獲取文件的修改時間
            file_time = os.path.getmtime(file_path)
            file_time_str = get_utc_time()

            # 分解文件路徑為相對路徑、文件名、文件擴展名和文件時間
            relative_path = os.path.relpath(file_path, project_path)
            file_name, file_extension = os.path.splitext(os.path.basename(file_path))
            load_file_path = os.path.dirname(relative_path)
            load_file_name = file_name
            load_file_subname = file_extension[1:]  # 去掉擴展名前面的點
            load_file_date = file_time_str
            print(f'load_file_path[{load_file_path}],\nload_file_name[{load_file_name}],\nload_file_subname[{load_file_subname}],\nload_file_date[{load_file_date}]')

            # 構建 GROUP 字段
            group_name = f'{load_file_name}-{load_file_date}.{load_file_subname}'

            # 連接到指定數據庫並插入數據
            conn = sqlite3.connect(db_path)
            cursor = conn.cursor()
            cursor.execute("SELECT FILE_NAME, FILE_PATH, FILE_DATE FROM CODE_GROUP")
            existing_files = cursor.fetchall()
            
            # 檢查文件是否已經存在於數據庫中
            if (load_file_name, load_file_path, load_file_date) in existing_files:
                print(f'文件 {file_path} 已存在於數據庫中')
            else:
                # 插入文件信息到 CODE_GROUP 表
                current_time = get_utc_time()
                
                cursor.execute("INSERT INTO CODE_GROUP (FILE_NAME, FILE_PATH, FILE_DATE, FILE_SUBNAME, GROUP_NAME, DATE) VALUES (?, ?, ?, ?, ?, ?)",
                            (load_file_name, load_file_path, load_file_date, load_file_subname, group_name, current_time))
                
                print(f'文件 {file_path} 的資料已插入數據庫')
                            
            # 中斷資料庫連線
            conn.commit()
            conn.close()

            # 將文件覆制到code_analysis文件夾下
            log_folder_path = os.path.join(project_path, "code_analysis", load_file_path )
            os.makedirs(log_folder_path, exist_ok=True)
            new_file_path = os.path.join(log_folder_path, f"{load_file_name}-{load_file_date}.{load_file_subname}")
            shutil.copy2(file_path, new_file_path)
            print(f'文件 {file_path} 已複製到 {new_file_path}')

        except Exception as e:
            print(f"Error: {e}")
    else:
        print(f'No Data:{file_path}')


# 根據需求複製新的程式版本
def copy_code_snippet(db_path, group_name, snippet_dynamic, snippet_transform, snippet_comment):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    try:
        # 查询符合条件的最新记录
        query = """
        SELECT GROUP_NAME, SNIPPET_NAME, SNIPPET_DYNAMIC, SNIPPET_CODE, SNIPPET_VERSION
        FROM CODE_SNIPPET
        WHERE GROUP_NAME = ? AND SNIPPET_DYNAMIC = ?
        ORDER BY SNIPPET_VERSION DESC
        LIMIT 1
        """
        cursor.execute(query, (group_name, snippet_dynamic))
        row = cursor.fetchone()

        if row is None:
            print('沒有取得任何符合資料')
            conn.close()
            return
        else:
            print(f'取得GROUP_NAME[{row[1]}]')

        new_row = list(row)
        new_row[0] = group_name
        new_row[1] = row[1]
        new_row[2] = snippet_dynamic
        new_row[3] = row[3]
        new_row[4] = row[4] + 1 if row[4] is not None else 1  # 版次增加1
        new_row.append(snippet_transform)
        new_row.append(snippet_comment)
        new_row.append(get_utc_time())

        insert_query = """
        INSERT INTO CODE_SNIPPET (GROUP_NAME, SNIPPET_NAME, SNIPPET_DYNAMIC, SNIPPET_CODE, SNIPPET_VERSION, SNIPPET_TRANSFORM, SNIPPET_COMMENT, DATE)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """
        cursor.execute(insert_query, new_row)
        conn.commit()
        
        print(f'完成{snippet_dynamic}複製，並生成新的變形{snippet_transform}')
    except Exception as e:
        print(f"Error: {e}")
    finally:
        conn.close()


# 組合 然後寫入
def process_snippets2(group_name, db_path):
    print('進入處理')

    # 取得資料
    data = []
    data = get_data_from_database(group_name, db_path)

    # 資料組合
    accumulated_data = accumulate_data(data)

    # 儲存資料
    save_data_to_database(group_name, accumulated_data, db_path)


# 動態資料處理
def process_dynamic_components(conn, group_name, snippet_data):
    cursor = conn.cursor()
    cursor.execute("DELETE FROM CODE_FUNCTION WHERE GROUP_NAME = ?", (group_name,))
    cursor.execute("SELECT SOURCE_ID, SOURCE_NAME, SOURCE_URL, SOURCE_URL2, SOURCE_HTML_KEYWORDS, SOURCE_HTML_DESCRIPTION, SOURCE_HTML_SUMMARY, SOURCE_HTML_CONTENT, SOURCE_SPECIAL_KEYWORDS, SOURCE_SPECIAL_CONTENT, SOURCE_CLEAN_CONTENT FROM DATA_SOURCE WHERE TRIM(SOURCE_URL2) <> ''")
    rows = cursor.fetchall()

    function_version = group_name + "_" + get_utc_time()

    for row in rows:
        source_id, source_name, source_url, source_url2, source_html_keywords, source_html_description, source_html_summary, source_html_content, source_special_keywords, source_special_content, source_clean_content = row

        build_function = ""
        build_function_comment = "/**\n/ Id:[{0}] Name:[{1}] Page:[{2}] \n*/\n".format(source_id, source_name, source_url)
        build_function += build_function_comment 
        build_function_name = "fn_" + source_url2
        build_function_special = "// Snippet Start:{0}\n{1}\n// Snippet End:{0}\n"
        build_function_dynamic = ""

        prefix_identify_keyword = "meta"

        if source_special_keywords and source_special_keywords is not None:
            build_function_dynamic = build_function_special.format("Dynamic keywords", source_special_keywords)
        elif source_html_keywords and source_html_keywords is not None:
            if source_html_keywords.startswith(prefix_identify_keyword):
                build_function_dynamic = get_code_snippet_from_snippet_data("Dynamic keywords", "metaContent", snippet_data) + "\n"
            else:
                build_function_dynamic = get_code_snippet_from_snippet_data("Dynamic keywords", "textContent", snippet_data) + "\n"
        else :
            build_function_dynamic = get_code_snippet_from_snippet_data("Dynamic keywords", "none", snippet_data) + "\n"
      
        if source_html_description and source_html_description is not None:
            if source_html_description.startswith(prefix_identify_keyword):
                build_function_dynamic += get_code_snippet_from_snippet_data("Dynamic description", "metaContent", snippet_data) + "\n"
            else:
                build_function_dynamic += get_code_snippet_from_snippet_data("Dynamic description", "textContent", snippet_data) + "\n"
        else:
            build_function_dynamic += get_code_snippet_from_snippet_data("Dynamic description", "none", snippet_data) + "\n"

        if source_html_summary and source_html_summary is not None:
            if source_html_summary.startswith(prefix_identify_keyword):
                build_function_dynamic += get_code_snippet_from_snippet_data("Dynamic summary", "metaContent", snippet_data) + "\n"
            else:
                build_function_dynamic += get_code_snippet_from_snippet_data("Dynamic summary", "textContent", snippet_data) + "\n"
        else:
            build_function_dynamic += get_code_snippet_from_snippet_data("Dynamic summary", "none", snippet_data) + "\n"

        if source_special_content and source_special_content is not None:
            build_function_dynamic += build_function_special.format("Dynamic content", source_special_content)
        elif source_html_content and source_html_content is not None:
            build_function_dynamic += get_code_snippet_from_snippet_data("Dynamic content", "", snippet_data) + "\n"
        else:
            build_function_dynamic += get_code_snippet_from_snippet_data("Dynamic content", "none", snippet_data) + "\n"

        build_function = get_code_snippet_from_snippet_data("Body", "",snippet_data).replace("fn_Test", build_function_name).replace("{CODE_DYNAMIC}", build_function_dynamic)

        
        write_to_code_function(conn, group_name, build_function_name, function_version, build_function)

        print(f'完成[{build_function_name}]生成')

    print(f'已處理{len(rows)}行資料')


# 分析程式並組件

## 分析程式

In [ ]:
# 增加專案

add_group_project( project_path, file_path, db_path)

In [ ]:
# 调用函数将代码写入数据库
        
write_code_to_database(file_path, db_path)


In [ ]:
# 初步對 CODE_STORE做資料

process_snippets1(group_name, db_path)

In [ ]:
# 轉譯複用 snippet 
process_snippets2(group_name, db_path)

In [ ]:
# 拷貝一筆 Snippet 做為新的 程式變形使用
snippet_dynamic = 'Dynamic content' # 拷貝目標
snippet_transform = 'none' # 變動新名稱
snippet_comment = '若為空，保留變數名稱' # 變動註解
copy_code_snippet(db_path, group_name, snippet_dynamic, snippet_transform, snippet_comment)

## 生成產品程式

In [ ]:
# 程式組合工廠
def combine_code_from_database(conn, snippet_data, group_name, output_folder, output_file_rename):
    try:
        combined_code = ""

        combined_code += get_code_snippet_from_snippet_data("Header", "", snippet_data) + "\n"

        cursor = conn.cursor()

        cursor.execute("""
            SELECT CODE
            FROM CODE_FUNCTION
            WHERE GROUP_NAME = ?
        """, (group_name,))
        rows = cursor.fetchall()

        for row in rows:
            code = row[0]
            combined_code += code + "\n"

        combined_code += get_code_snippet_from_snippet_data("Footer", "", snippet_data) + "\n"

        formatted_code = jsbeautifier.beautify(combined_code)

        output_file_name = group_name
        if output_file_rename:
            output_file_name = output_file_name.replace(group_name, output_file_rename)

        output_path = os.path.join(output_folder, output_file_name)

        with codecs.open(output_path, 'w', encoding='utf-8') as file:
            file.write(formatted_code)

        print(f"程式碼已成功生成：{output_path}")

    except Exception as e:
        print(f"Error：{e}")

# 處理資料庫資料，並生成主程式
def build_function_code(group_name, db_path, output_folder, output_file_rename):
    conn = connect_database(db_path)

    snippet_data = get_code_snippet_from_database(conn, group_name)

    combine_code_from_database(conn, snippet_data, group_name, output_folder, output_file_rename)

    close_database(conn)

build_function_code(group_name, db_path, output_folder, output_file_rename)